In [1]:
import numpy as np
import pandas as pd
import glob
import tensorflow as tf
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.utils import to_categorical

In [3]:
gpu=tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0],True)

In [4]:
path=glob.glob('data/*')
x=[]
y=[]

Lấy mẫu 10 frame 1 lần

In [5]:
for idx,ppath in enumerate(path):
    df=pd.read_csv(ppath)
    dataset=df.iloc[:,1:].values
    for j in range(0,len(dataset),15):
        if j+15>len(dataset):break
        x.append(dataset[j:15+j])
        y.append(idx)

In [6]:
onehot=to_categorical(y).astype(int)

In [7]:
onehot.shape

(1120, 28)

In [8]:
x=np.array(x)
x.shape
# (130, 10, 60) 
# 130 mẫu, 10 frame, 60 landmarks trong 10 frame
# 1 frame có 20 landmarks do bỏ điểm 0, trong 1 frame có 3 tọa độ x,y,z => 3*30 = 60

(1120, 15, 60)

https://www.phamduytung.com/blog/2019-02-06-choosing-the-right-hyperparameters-for-a-simple-lstm-using-keras/#:~:text=alpha%20*%20(N_i%20%2B%20N_o))%7D-,N,d%E1%BB%AF%20li%E1%BB%87u%20m%E1%BA%ABu%2C%20th%C3%AC%20con%20s%E1%BB%91%20c%C3%B3%20v%E1%BA%BB%20h%E1%BB%A3p%20l%C3%BD%20h%C6%A1n.,-Trong%20t%E1%BA%ADp%20d%E1%BB%AF

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, onehot, test_size=0.2)

model  = Sequential()
model.add(LSTM(units = 56, return_sequences = True, input_shape = (15, 60))) # input_shape = 15 frame, 1 frame has 60 landmarks
model.add(Dropout(0.2))
model.add(LSTM(units = 56, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 56, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 56, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 56))
model.add(Dropout(0.1))
model.add(Dense(units = 28, activation="softmax"))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()
model.fit(X_train, y_train, epochs=100, batch_size=30,validation_data=(X_test, y_test))
# model.save("model.h5")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 56)            26208     
                                                                 
 dropout (Dropout)           (None, 15, 56)            0         
                                                                 
 lstm_1 (LSTM)               (None, 15, 56)            25312     
                                                                 
 dropout_1 (Dropout)         (None, 15, 56)            0         
                                                                 
 lstm_2 (LSTM)               (None, 15, 56)            25312     
                                                                 
 dropout_2 (Dropout)         (None, 15, 56)            0         
                                                                 
 lstm_3 (LSTM)               (None, 15, 56)            2

In [10]:
res=model.predict(X_test)
np.argmax(res[2])

7/7 [==============================] - 3s 18ms/step


5

In [11]:
np.argmax(res[2]),y_test[2,np.argmax(res[2])]

(5, 1)

In [12]:
model.save('model.h5')

In [13]:
from sklearn.metrics import multilabel_confusion_matrix,confusion_matrix,accuracy_score


In [14]:
y_pre=np.zeros_like(res)

In [15]:
y_pre[np.arange(len(res)),res.argmax(1)]=1
y_pre

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
multilabel_confusion_matrix(y_test,y_pre)

array([[[218,   0],
        [  0,   6]],

       [[219,   0],
        [  0,   5]],

       [[214,   0],
        [  0,  10]],

       [[212,   2],
        [  0,  10]],

       [[216,   0],
        [  0,   8]],

       [[217,   0],
        [  0,   7]],

       [[223,   0],
        [  0,   1]],

       [[214,   3],
        [  0,   7]],

       [[219,   0],
        [  0,   5]],

       [[217,   0],
        [  0,   7]],

       [[216,   0],
        [  0,   8]],

       [[214,   0],
        [  0,  10]],

       [[216,   0],
        [  0,   8]],

       [[220,   0],
        [  0,   4]],

       [[218,   0],
        [  0,   6]],

       [[213,   0],
        [  0,  11]],

       [[212,   0],
        [  2,  10]],

       [[214,   0],
        [  0,  10]],

       [[213,   6],
        [  0,   5]],

       [[212,   0],
        [  0,  12]],

       [[216,   0],
        [  1,   7]],

       [[215,   0],
        [  0,   9]],

       [[211,   2],
        [  7,   4]],

       [[214,   0],
        [  3, 

In [17]:
def test(y_hat, test_y):
    """test
    Compute the confusion matrix based on labels and predicted values 

    :param y_hat: predicted probabilites, output of classifier.feed_forward
    :param test_y: test labels
    """
    if (y_hat.ndim == 2):
        y_hat = np.argmax(y_hat, axis=1)
    num_class = np.unique(test_y).size
    confusion_mat = np.zeros((num_class, num_class))

    for i in range(num_class):
        class_i_idx = test_y == i
        num_class_i = np.sum(class_i_idx)
        y_hat_i = y_hat[class_i_idx]
        for j in range(num_class):
            confusion_mat[i,j] = 1.0*np.sum(y_hat_i == j)/num_class_i

    np.set_printoptions(precision=2)
    print('Confusion matrix:')
    print(confusion_mat)
    print('Diagonal values:')
    print(confusion_mat.flatten()[0::(num_class+1)])

In [33]:
multilabel_confusion_matrix(y_test,y_pre)
# array([[[218,   0],               input = 1120 mẫu, lấy 0.2 = 20% mẫu để test
#         [  0,   6]],              ==> có 224 mẫu test
#                                   Nhận xét array[0] đầu tiên: có tổng số 224 mẫu
#        [[219,   0],               với 218 mẫu dự đoán đúng, thực tế đúng,
#         [  0,   5]],              6 mẫu dự đoán sai thực tế sai

#        [[214,   0],
#         [  0,  10]],

#        [[212,   2],
#         [  0,  10]],

array([[[218,   0],
        [  0,   6]],

       [[219,   0],
        [  0,   5]],

       [[214,   0],
        [  0,  10]],

       [[212,   2],
        [  0,  10]],

       [[216,   0],
        [  0,   8]],

       [[217,   0],
        [  0,   7]],

       [[223,   0],
        [  0,   1]],

       [[214,   3],
        [  0,   7]],

       [[219,   0],
        [  0,   5]],

       [[217,   0],
        [  0,   7]],

       [[216,   0],
        [  0,   8]],

       [[214,   0],
        [  0,  10]],

       [[216,   0],
        [  0,   8]],

       [[220,   0],
        [  0,   4]],

       [[218,   0],
        [  0,   6]],

       [[213,   0],
        [  0,  11]],

       [[212,   0],
        [  2,  10]],

       [[214,   0],
        [  0,  10]],

       [[213,   6],
        [  0,   5]],

       [[212,   0],
        [  0,  12]],

       [[216,   0],
        [  1,   7]],

       [[215,   0],
        [  0,   9]],

       [[211,   2],
        [  7,   4]],

       [[214,   0],
        [  3, 

In [29]:
y_pre.shape

(224, 28)

In [22]:
y_test.shape

(224, 28)